In [1]:
# Parameters
num_cores = 27
mem = 650000
config = "configs.config_Reddit"


In [2]:
### The steps of preprocessing:

# 1- cleaning
# 2- Unigram
#     2-1 - English
#     2-2 - Stop Words
#     2-3 - Stemming
# 3- Bigram
# 4- Tokenization
# 6- Lemmatization
# 7- Thresholding: 
#     7-1 min < word < % of corupus
#     7-2 min_words_per_docs
# 8- LDA

### The steps of preprocessing for tokens:

# 1- cleaning
# 2- Stemming

In [3]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_SE_test_local'
#     config = 'configs.config_Reddit_test_local'

In [4]:
### initialization ###

from _imports import *
# from _utils import *

print('config file is set to {}'.format(config))

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_Reddit
num_cores is set to 27
mem is set to 650000


In [5]:
### Functions for decoding and date conversion ###


def parallel_decode(list_tags):
    try:
#         return ', '.join(list(map(util.decode_tag, df)))
        return ', '.join([util.decode_tag(tag.strip(), c) for tag in list_tags.split(',')])    
    except Exception as e:
#         text = ', '.join(list(list_tags))
        util.print_asynch(list_tags)
        raise util.Failed('couldn\'t find {}\n{}!'.format(list_tags), e)

def parallel_decode_df(df):
    return df.apply(parallel_decode)


def parallel_decode_full(df):
    try:
#         return ', '.join(list(map(util.decode_tag, df)))
        return ', '.join([util.decode_tag(tag.strip(), c) for tag in df['txt_lemmatized'].split(',')])    
    except Exception as e:
        util.print_asynch(df)
        raise util.Failed('couldn\'t find {}\n{}!'.format(df), e)


def parallel_decode_df_full(df):
    return df.apply(parallel_decode_full, axis=1)


def str_timestamp_to_date(date):
    try:
        return datetime.fromtimestamp(int(date))
    except ValueError as e:
#         util.count_sync(10)
        raise Failed('{} couldn\'t be converted. \n {}'.format(date, e))

def str_timestamp_to_date_df(df):
    return df.apply(str_timestamp_to_date)

In [6]:
### concatenating the dataframes + decoding the keywords + saving ###


# File loading

if c.tags is not None:
    df_tags = pd.read_csv('{}{}_0_tags.csv'.format(c.directory['save'], c.project_name))
    c.tags_dict = dict(zip(df_tags['key'], df_tags['val']))


df_text_concat = pd.read_csv('{}{}_3_text_lemmatized_tagged.csv'.format(c.directory['save'], c.project_name))
df_cols = list(df_text_concat.columns)


df_text_concat['date']=util.parallelize_df(df_text_concat['date'], str_timestamp_to_date_df, c)
        
# c.init_parallel(True, c)
df_text_concat = pd.concat([
    df_text_concat,
#     df_text_concat['txt_lemmatized'].parallel_apply(para_decode),
#     util.parallelize_df(df_text_concat['txt_lemmatized'], parallel_decode_df, c),
    util.parallelize_df(df_text_concat, parallel_decode_df_full, c),
#     util.parallelize_df_dask(df_text_concat['txt_lemmatized'], parallel_decode_df, c),    
],
    ignore_index=True,
    axis=1
)
df_cols.extend(['keywords'])
df_text_concat.columns = df_cols
df_text_concat.to_csv('{}{}_4_text_lemmatized_decoded.csv'.format(c.directory['save'], c.project_name),
                      encoding='utf-8',
                      index=False
                      )

try:
    os.remove('{}{}_sql_docs_decoded.sqlite'.format(c.directory['save'],c.project_name))
except:
    pass

util.save_df_to_sql(df_text_concat,
                   '{}{}_sql_docs_decoded.sqlite'.format(c.directory['save'],c.project_name),
                    'docs'
                   )
pprint(df_text_concat.shape)
util.display_df(df_text_concat.head())

parallel: 27 partitions with 27 cores for str_timestamp_to_date_df
parallel: 27 partitions with 27 cores for parallel_decode_df_full
(2225657, 8)


,id,date,category,all_text,group,txt_orig,txt_lemmatized,keywords
0,2qzibe,2015-01-01 00:29:40,MachineLearning,Measuring human performance on standard image ...,submissions,Measuring human performance on standard image ...,"measur, human, zzzltwolfivelonelzzz, standard,...","measur, human, perform, standard, imag classif..."
1,2qzv35,2015-01-01 03:52:22,dataisbeautiful,European economy guide: Taking Europe’s pulse,submissions,European economy guide: Taking Europe’s pulse,"european, economi, guid, take, europ, pul","european, economi, guid, take, europ, pul"
2,2qzzaa,2015-01-01 05:23:25,dataisbeautiful,Popularity of visual forms in DataIsBeautiful ...,submissions,Popularity of visual forms in DataIsBeautiful ...,"popular, zzzlthreelzerolfourlzzz, form, datais...","popular, visual, form, dataisbeauti, base, sam..."
3,2r048c,2015-01-01 07:16:15,dataisbeautiful,JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,submissions,JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,"joe, groom, daili, shampoo, good, shampoo, oil...","joe, groom, daili, shampoo, good, shampoo, oil..."
4,2r0e7b,2015-01-01 10:25:46,dataisbeautiful,Los Angeles Traffic Accident Rate in Rainy vs ...,submissions,Los Angeles Traffic Accident Rate in Rainy vs ...,"trafficaccid, rate, raini, dri, weather","trafficaccid, rate, raini, dri, weather"
